# FinBERT Example Notebook

This notebooks shows how to train and use the FinBERT pre-trained language model for financial sentiment analysis.

## Modules 

In [1]:
from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
import finbert.utils as tools

%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', -1)

<ipython-input-1-ce2e9558c130>:21: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


# Get predictions

With the `predict` function, given a piece of text, we split it into a list of sentences and then predict sentiment for each sentence. The output is written into a dataframe. Predictions are represented in three different columns: 

1) `logit`: probabilities for each class

2) `prediction`: predicted label

3) `sentiment_score`: sentiment score calculated as: probability of positive - probability of negative

Below we analyze a paragraph taken out of [this](https://www.economist.com/finance-and-economics/2019/01/03/a-profit-warning-from-apple-jolts-markets) article from The Economist. For comparison purposes, we also put the sentiments predicted with TextBlob.
> Later that day Apple said it was revising down its earnings expectations in the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours trading and the decline was extended to more than 10% when the market opened. The dollar fell by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. Yields on government bonds fell as investors fled to the traditional haven in a market storm.

In [2]:
df = pd.read_csv("scrap_table.csv")

cl_path = project_dir/'finbert'/'models'/'classifier_model'/'finbert-sentiment'
model = AutoModelForSequenceClassification.from_pretrained(cl_path, cache_dir=None, num_labels=3)

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/kakao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
results = pd.DataFrame()
for i in range(20):
    text = df.loc[i,'title']
    result = predict(text, model)
    blob = TextBlob(text)
    result['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]
    results = results.append(result, ignore_index=True)

 0 0 0 0 0 0 0 0 0 0 0
06/05/2021 22:45:00 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
06/05/2021 22:45:00 - INFO - finbert.utils -   label: None (id = 9090)
06/05/2021 22:45:01 - INFO - root -   tensor([[-1.9169, -0.8279,  2.3887]])
06/05/2021 22:45:05 - INFO - finbert.utils -   *** Example ***
06/05/2021 22:45:05 - INFO - finbert.utils -   guid: 0
06/05/2021 22:45:05 - INFO - finbert.utils -   tokens: [CLS] apple ( aa ##pl ) upgraded to strong buy : here ' s why [SEP]
06/05/2021 22:45:05 - INFO - finbert.utils -   input_ids: 101 6207 1006 9779 24759 1007 9725 2000 2844 4965 1024 2182 1005 1055 2339 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
06/05/2021 22:45:05 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [4]:
results

,sentence,logit,prediction,sentiment_score,textblob_prediction
0,"If You Own Apple (AAPL) Stock, Should You Sell It Now?","[0.012807425, 0.038053326, 0.9491393]",neutral,-0.025246,0.600000
1,Apple (AAPL) Upgraded to Strong Buy: Here's Why,"[0.8442952, 0.009305688, 0.14639913]",positive,0.834989,0.433333
2,Apple (AAPL): Bull of the Day,"[0.8572408, 0.022612732, 0.12014642]",positive,0.834628,0.000000
3,Apple Inc.'s (NASDAQ:AAPL) Share Price Not Quite Adding Up,"[0.042672113, 0.8986128, 0.058715057]",negative,-0.855941,0.000000
4,Apple Inc (AAPL) CEO Timothy D Cook Sold $131.8 million of Shares,"[0.011218424, 0.032685027, 0.9560966]",neutral,-0.021467,0.000000
5,"Apple (AAPL) Has Risen 98% in Last One Year, Outperforms Market","[0.96862966, 0.019872326, 0.011498033]",positive,0.948757,0.000000
6,Brace Yourself for a Sharp Rise in Inflation,"[0.14527018, 0.29884762, 0.5558822]",neutral,-0.153577,-0.125000
7,"Tesla Gets $2,500 3-Year Target From Munster, But Analyst Sees Apple As A Roadblock In EV Maker's Path","[0.2404817, 0.3526927, 0.40682557]",neutral,-0.112211,0.000000
8,Exclusive: Apple targets car production by 2024 and eyes 'next level' battery technology - sources,"[0.017024903, 0.013253736, 0.96972144]",neutral,0.003771,0.000000
9,Apple reportedly working on an electric car sends the stock of this Bill Gates-backed battery startup surging,"[0.021447763, 0.94124424, 0.037307967]",negative,-0.919796,0.000000


In [5]:
results.to_csv("result.csv", header=True, index=False)